# Install libs

In [1]:
!pip install --upgrade --quiet langchain langchain-community sentence-transformers faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# File Uploading

In [3]:
from google.colab import files
uploaded = files.upload()

Saving About_me.txt to About_me.txt


In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("About_me.txt")
documents = loader.load()

print("Loaded successfully!")
print(documents[0].page_content[:300])

Loaded successfully!
PERSONAL INFORMATION
Name: Ahmed Mohamed
Age: 21
Location: Cairo, Egypt
Email: ahmed.mohamed@email.com
LinkedIn: linkedin.com/in/ahmed-mohamed

EDUCATIONAL BACKGROUND
Degree: Bachelor of S


# Text Splitting

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

chunks = splitter.split_documents(documents)
print("Chunks:", len(chunks))

Chunks: 26


# Embedding

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("Embeddings ready!")

/tmp/ipython-input-2981112155.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings ready!


# FAISS Vector Store

In [7]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

print("Vector DB ready!")

Vector DB ready!


# LLM Setup

In [8]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline("text-generation", model="google/flan-t5-base")  # correct task
llm = HuggingFacePipeline(pipeline=pipe)

print("LLM ready!")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'ExaoneMoeForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalL

LLM ready!


/tmp/ipython-input-3093924496.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


# Testing (Ask Questions)

In [17]:
def ask(question):
    print(f"Question: {question}")

    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)

    prompt = f"""Answer the question using only the context below.

Context:
{context}

Question: {question}
Answer:"""

    # Get answer
    result = llm.invoke(prompt)
    print(f"Answer: {result}")
    print("="*60)

ask("What are this person's programming skills?")
ask("Where did this person study?")
ask("What projects has this person worked on?")
ask("What are this person's future goals?")

Question: What are this person's programming skills?


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: Answer the question using only the context below.

Context:
TECHNICAL SKILLS
Programming Languages: Python, C++, SQL, JavaScript
Machine Learning: Scikit-learn, XGBoost, Random Forest, SVM, Linear Regression, Logistic Regression
Deep Learning: TensorFlow, Keras, PyTorch

SOFT SKILLS
- Strong problem-solving and analytical thinking
- Excellent communication and teamwork
- Fast learner and self-motivated
- Time management and ability to work under pressure
- Leadership and project management

Question: What are this person's programming skills?
Answer:
Question: Where did this person study?


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: Answer the question using only the context below.

Context:
EDUCATIONAL BACKGROUND
Degree: Bachelor of Science in Computer Science
University: Cairo University, Faculty of Computers and Artificial Intelligence
Expected Graduation: 2025
GPA: 3.6 / 4.0

Expected Graduation: 2025
GPA: 3.6 / 4.0
Relevant Courses: Data Structures, Algorithms, Machine Learning, Natural Language Processing, 
Database Systems, Deep Learning, Computer Vision, Software Engineering, Statistics and Probability.

Question: Where did this person study?
Answer:
Question: What projects has this person worked on?


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: Answer the question using only the context below.

Context:
PROJECTS

- Collaborated with the team on real-world AI projects

Question: What projects has this person worked on?
Answer:
Question: What are this person's future goals?


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: Answer the question using only the context below.

Context:
GOALS AND ASPIRATIONS
Short-term Goals:
- Graduate with honors from Cairo University
- Land a full-time NLP Engineer or ML Engineer role at a top tech company
- Publish at least one research paper in NLP

SOFT SKILLS
- Strong problem-solving and analytical thinking
- Excellent communication and teamwork
- Fast learner and self-motivated
- Time management and ability to work under pressure
- Leadership and project management

Question: What are this person's future goals?
Answer:
